In [1]:
import numpy as np
import cv2

In [2]:
import os

In [3]:
filepre = 'data/1'
imnames = os.listdir(filepre)

In [258]:
def show(image, windowname = 'image'):
    cv2.imshow(windowname, image)
    keypress = cv2.waitKey(0)
    if keypress:
        cv2.destroyWindow(windowname)

In [255]:
from numpy.linalg import norm

def canny_norm_filter(image):
    im = norm(image, axis = -1)/((3*(255**2))**.5)
    return cv2.Canny( (im*255).astype(np.uint8), 100, 150)

def canny_color_filter(image):
    return cv2.Canny(image, 100, 150)

def downsample_filter(image, by = 5):
    return image[::by,::by,:]

def difference_kernel(image, size = 3):
    im = np.moveaxis(image, -1, 0)

    size = 3
    gradient_kernel = -np.ones([size, size])
    gradient_kernel /= size**2
    gradient_kernel[ int(size/2) ][ int(size/2) ] *= -1
    
    im = np.stack(correlate2d(im[a], gradient_kernel, boundary = 'symm') for a in range(3) )

    return np.moveaxis(im, 0, -1).astype(np.uint8)

In [256]:
def video_filter(*image_pipeline):
    cap = cv2.VideoCapture(1)
    cv2.namedWindow('Main', cv2.WINDOW_NORMAL)
        
    while cap.isOpened():
        ret, frame = cap.read()
        
        for filter in image_pipeline:
            frame = filter(frame)
        
        cv2.imshow('Main', frame)
        
        keypress = cv2.waitKey(1)
        if keypress == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

In [257]:
video_filter(downsample_filter, canny_norm_filter)